# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 19:17:05 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658633,35.2,1454651,77.7,1227761,65.6
Vcells,1222301,9.4,8388608,64.0,1975161,15.1


In [2]:
plocal <- list()

# 501
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 500L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.4
plocal$num_iterations <- 64

plocal$APO <- 1
plocal$ksemillerio <- 50


In [4]:
source("/home/vickydiliscia/dmeyf2025/src/churn/C3020/config_100271/_index.R")

In [5]:
source(file.path(PARAM$paths$functions, "index.R"))
source_functions(PARAM$paths$functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: agregar_var_rf.R

Sourcing: aplicar_dataquality_zeros.R

Sourcing: apply_sum_specs_multi.R

Sourcing: audit_nullear.R

Sourcing: build_variables_rotas.R

Sourcing: crear_proporcion.R

Sourcing: deflactar_por_ipc.R

Sourcing: features_historicos_optimizados.R

Loading required package: Rcpp

Sourcing: features_historicos.R

Sourcing: flip.R

Sourcing: interpolar_faltantes.R

Sourcing: lags.R

Sourcing: normalizar_ctrx_quarter.R

Sourcing: print_utils.R

Sourcing: run_features_historicas.R

Sourcing: tendencias_y_mas.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [6]:
PARAM$experimento

[1] "APO3020-202107_100271"

In [7]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [8]:
paths <- c(PARAM$paths$datasetcrudo2,
           PARAM$paths$datasetcrudo3)

dataset <- rbindlist(
  lapply(paths, fread),
  use.names = TRUE,
  fill      = TRUE
)

dataset <- agregar_clase_ternaria(dataset)
setorder(dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, .(foto_mes, clase_ternaria)]
Sys.time()

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


[1] "2025-12-03 19:17:53 UTC"

### Eliminacion de Features

In [9]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]
Sys.time()

[1] "2025-12-03 19:17:53 UTC"

### Data Quality

#### Flip Internet

In [10]:
flip(dataset, internet, month_to = 202009L)
Sys.time()

[1] "2025-12-03 19:17:54 UTC"

#### Tratamiento variables con meses rotos

In [11]:
variables_rotas <- build_variables_rotas(dataset)

if (PARAM$dataquality$zeros == "nullear") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "nullear"
  )

} else if (PARAM$dataquality$zeros == "interpolar") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "interpolar",
    treat_zero_as_na = TRUE,
    require_both     = TRUE,
    only_if_na       = TRUE,
    integer_prefix   = "^c",
    round_integers   = TRUE
  )

} else {
  message("No se aplicó tratamiento de dataquality.")
}

Sys.time()

[1] "2025-12-03 19:18:06 UTC"

In [12]:
if (PARAM$dataquality$zeros == "nullear") {
# Si usaste replace_all_in_month = FALSE (nullear solo ceros):
res <- audit_nullear(dataset, variables_rotas, month_col = "foto_mes",
                     replace_all_in_month = FALSE)

# Ver primeras filas y variables con problemas (si las hubiera)
res[ok_sin_ceros_marc | status == "ok"][]
}else{
    message("No se audita")
}
Sys.time()

variable,tipo,meses_marcados,filas_marcadas,ceros_en_marcados,NA_en_marcados,ceros_fuera,NA_fuera,ok_sin_ceros_marc,modo,status
<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<lgl>,<chr>,<chr>
active_quarter,integer,202006,153102,0,153102,66713,0,TRUE,nullear_solo_ceros,ok
catm_trx,integer,202006,153102,0,153102,2694879,0,TRUE,nullear_solo_ceros,ok
catm_trx_other,integer,202006,153102,0,153102,3552366,0,TRUE,nullear_solo_ceros,ok
ccajas_consultas,integer,202006,153102,0,153102,4393036,0,TRUE,nullear_solo_ceros,ok
ccajas_depositos,integer,"202006,202105",317019,0,317019,4274594,0,TRUE,nullear_solo_ceros,ok
ccajas_extracciones,integer,202006,153102,0,153102,4200898,0,TRUE,nullear_solo_ceros,ok
ccajas_otras,integer,202006,153102,0,153102,4592804,0,TRUE,nullear_solo_ceros,ok
ccajas_transacciones,integer,202006,153102,0,153102,3897747,0,TRUE,nullear_solo_ceros,ok
ccajeros_propios_descuentos,integer,"201910,202002,202006,202009,202010,202102",914901,0,914901,3795639,0,TRUE,nullear_solo_ceros,ok


[1] "2025-12-03 19:18:13 UTC"

## Data Drifting

In [13]:
library(data.table)

if (isTRUE(PARAM$FE_hist$Ipc$Run)) {
    ipc_dt <- fread(PARAM$FE_hist$specsipc)
    
    dataset <- deflactar_por_ipc(
        dt = dataset,
        ipc_dt = ipc_dt,
        replace = TRUE,
        base_mes = 202108L 
        )
    message(paste0("IPC aplicado con base ", 202109L ))
} else {
    message(paste0("Tecnica de data drifting no aplicada", 202109L ))
}
Sys.time()

IPC aplicado con base 202109



[1] "2025-12-03 19:18:14 UTC"

### Feature Engineering Intra-Mes

#### Variables Sugeridas

In [14]:
if (isTRUE(PARAM$FE_intrames$kmes)) {
  dataset[, kmes := foto_mes %% 100]
  message("Columna kmes creada")
} else {
  message("Columna kmes no creada")
}

if (isTRUE(PARAM$FE_intrames$normquarter)) {
  dataset <- normalizar_ctrx_quarter(dataset)
  message("Normalizacion ctrx_quarter aplicada")
} else {
  message("Normalizacion ctrx_quarter no aplicada")
}

if (isTRUE(PARAM$FE_intrames$ratio_payroll_edad)) {
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]  
    message("Columna mpayroll_sobre_edad creada")
} else {
    message("Columna mpayroll_sobre_edad creada")
}

Sys.time()

Columna kmes creada

Normalizacion ctrx_quarter aplicada

Columna mpayroll_sobre_edad creada



[1] "2025-12-03 19:18:14 UTC"

#### Sumas de Variables

In [ ]:
spec_sumas <- fread(PARAM$FE_intrames$specsumas)

# Normalizar nombres de columnas
setnames(spec_sumas, tolower(names(spec_sumas)))
req <- c("new_var","inputs","active")
faltan <- setdiff(req, names(spec_sumas))
if (length(faltan)) stop("Faltan columnas en spec: ", paste(faltan, collapse=", "))

# Limpiar espacios en inputs y new_var
spec_sumas[, new_var := trimws(new_var)]
spec_sumas[, inputs  := trimws(inputs)]

# 2) Convertir 'active' a lógico robusto
to_bool <- function(x){
  if (is.logical(x)) return(x)
  x2 <- tolower(trimws(as.character(x)))
  x2 %in% c("true","t","1","yes","y","si","sí")
}
spec_sumas[, active := to_bool(active)]

dataset <- apply_sum_specs_multi(
  dataset,
  spec = spec_sumas,
  from_file = FALSE,
  sep_inputs = ","
)
Sys.time()

✅ 'mtarjeta_total_consumo' = suma_segura(mtarjeta_visa_consumo+mtarjeta_master_consumo)

✅ 'mtarjeta_total_debitos_automaticos' = suma_segura(mttarjeta_visa_debitos_automaticos+mttarjeta_master_debitos_automaticos)



### Feature Engineering Historico

#### Lags y Delta Lags

In [ ]:
setorder(dataset, numero_de_cliente, foto_mes)

if (isTRUE(PARAM$FE_hist$Lags$Run)) {
  dataset <- crear_lags(dataset, lags = PARAM$FE_hist$Lags$Order)
} else {
  message("Lags no aplicados")
}

if (isTRUE(PARAM$FE_hist$DeltaLags$Run)) {
  dataset <- crear_delta_lags(dataset, lags = PARAM$FE_hist$DeltaLags$Order)
} else {
  message("Delta Lags no aplicados")
}

Sys.time()

#### Tendencias 6 meses

In [ ]:
# 1) ¿Es data.table?
is.data.table(dataset)

# 2) ¿Qué columnas está intentando laguear?
id_col <- "numero_de_cliente"; time_col <- "foto_mes"
exclude <- c("clase_ternaria")

cols_lagueables <- setdiff(names(dataset), c(id_col, time_col, exclude))
length(cols_lagueables); head(cols_lagueables)

# 3) ¿Hay más de un registro por cliente (si no, los lags serían todo NA)?
dataset[, .N, by = id_col][N > 1L][1:5]

In [ ]:

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    tendencias_y_mas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, 
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [26]:
plocal

$qcanaritos
[1] 5

$min_data_in_leaf
[1] 500

$learning_rate
[1] 1

$gradient_bound
[1] 0.4

$num_iterations
[1] 64

$APO
[1] 1

$ksemillerio
[1] 50

In [27]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [28]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-12-03 16:46:25 UTC"

#### Registros cambio las proporciones de POS/NEG

In [29]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [30]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [31]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= plocal$num_iterations, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-12-03 16:46:26 UTC"

In [32]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000)

In [33]:
detach("package:lightgbm", unload= TRUE)

In [34]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm

Registered S3 methods overwritten by 'zlightgbm':
  method                 from    
  dimnames<-.lgb.Dataset lightgbm
  dim.lgb.Dataset        lightgbm
  dimnames.lgb.Dataset   lightgbm
  predict.lgb.Booster    lightgbm
  print.lgb.Booster      lightgbm
  summary.lgb.Booster    lightgbm



In [35]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [36]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 827009 909409 436061 917729 237203 258283 156329 604759 527897 504967
[11] 840941 906329 304631 489871 193751 728899 897067 843907 360197 723553
[21] 972557 937991 623983 529517 234473 398509 237619 674789 909599 159773
[31] 151531 703249 442769 534553 257171 613451 914813 246497 680353 519119
[41] 233419 362233 752251 141677 302221 400277 450361 253439 206651 952481

In [37]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [38]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 237459 columnas 1505 


[1] "2025-12-03 16:46:35 UTC"

In [39]:
getwd()

[1] "/home/vickydiliscia/buckets/b1/exp/APO3002-202107_100271"

In [40]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-12-03 16:58:50 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [41]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [42]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [43]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [44]:
# aplico el modelo a los datos del future
dir.create("predicciones_modelitos", showWarnings = FALSE)

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
        
      vpred <- predict(modelo_final, mfuture)   # <- guardo primero la prediccion individual
      fwrite(
        data.table(
        numero_de_cliente = dfuture$numero_de_cliente,
        prob              = vpred
      ), file = sprintf("predicciones_modelitos/prediccion_semilla_%s.txt", sem)
      )
        
      # ---- luego acumulo al ensemble ----
      vpred_acum <- vpred_acum + vpred
      qacumulados <- qacumulados + 1

      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-12-03 17:01:24 UTC"

In [45]:
mganancias

4.2e+08,425200000,425600000,427600000,430400000,428400000,425600000,4.26e+08,424800000


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [46]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [47]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 11000

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-12-03 17:02:27 UTC"

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [48]:
colmedias <- colMeans( mganancias, na.rm=TRUE )
mcorte_mejor <- max(colmedias, na.rm=TRUE)
icorte_mejor <- which.max( colmedias )
corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [49]:
tbl <- as.data.table( as.list( colmedias ) )
colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
tbl[, experimento := PARAM$experimento ]

exp_gral <- "/content/buckets/b1/exp/apo-gral"
dir.create(exp_gral, showWarnings=FALSE)
fwrite( tbl,
  file= paste0( exp_gral, "/tb_experimentos.txt"),
  sep= "\t",
  append= TRUE
)

In [50]:
colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
tbl_local <- as.data.table( mganancias )

fwrite( tbl_local,
  file= "tb_apo.txt",
  sep= "\t"
)

In [51]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

mcorte_mejor
icerca
tb_prediccion[ icerca]

[1] 430400000

[1] 11000

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
784777919,202107,1,0.3721413,430400000


In [52]:
icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
icerca

[1] 11000

In [53]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:icerca, Predicted := 1L] # marco los primeros

archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_",  icerca, ".csv")

# grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_pseudo_kaggle,
  sep= ","
)


In [54]:
# la subida a Kaggle

In [55]:
Sys.time()

[1] "2025-12-03 17:02:33 UTC"

# Entrega Final Final

In [56]:
PARAM$train_entrega

$future
[1] 202109

$undersampling
[1] 0.05

$meses
 [1] 201901 201902 201903 201904 201905 201906 201907 201908 201909 201910
[11] 201911 201912 202001 202002 202003 202004 202005 202007 202008 202009
[21] 202010 202011 202012 202101 202102 202103 202104 202105 202106 202107

$training
 [1] 201901 201902 201903 201904 201905 201906 201907 201908 201909 201910
[11] 201911 201912 202001 202002 202003 202004 202005 202007 202008 202009
[21] 202010 202011 202012 202101 202102 202103 202104 202105 202106 202107

In [57]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_entrega$meses]

In [58]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_entrega$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

In [59]:
dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [60]:


cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(plocal$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-12-03 17:02:44 UTC"

In [61]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [62]:
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 258121 columnas 1505 


[1] "2025-12-03 17:02:48 UTC"

In [66]:
# genero los modelitos
dir.create( "modelitos_produccion", showWarnings= FALSE)


for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos_produccion/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-12-03 17:17:39 UTC"

In [67]:
# 1) Datos del future (entrega)
dfuture <- dataset[ foto_mes %in% PARAM$train_entrega$future ]

filas <- nrow(dfuture)

# 2) Agrego canaritos (solo una vez)
for( i in seq(plocal$qcanaritos) ){
  dfuture[, paste0("canarito_", i) := runif(filas) ]
}

# 3) Matriz de features para predict (sin clase)
mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

# 4) Carpeta donde voy a guardar las predicciones por semilla
dir.create("predicciones_produccion", showWarnings = FALSE)

# 5) Loop por semillas: cargo modelo, predigo y guardo
for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos_produccion/mod_", sem, ".txt")

  if( file.exists(arch_modelo) ) {

    modelo_final <- lgb.load(arch_modelo)

    prediccion <- predict(
      modelo_final,
      mfuture
    )

    # guardo SOLO numero_de_cliente y prob
    fwrite(
      data.table(
        numero_de_cliente = dfuture$numero_de_cliente,
        prob              = prediccion
      ),
      file = sprintf("predicciones_produccion/pred_semilla_%s.csv", sem)
    )

    rm(modelo_final, prediccion)
    gc()
  }
}


## Archivo de entrega

In [68]:
library(data.table)

# 1) Listar todos los archivos de predicciones por semilla
pred_files <- list.files(
  "predicciones_produccion",
  pattern = "^pred_semilla_.*\\.csv$",
  full.names = TRUE
)

if (length(pred_files) == 0L) {
  stop("No se encontraron archivos pred_semilla_*.csv en 'predicciones_produccion/'")
}

# 2) Leer todos los archivos
pred_list <- lapply(pred_files, fread)

# 3) Construir matriz de probabilidades (una columna por semilla)
probs_mat <- sapply(pred_list, `[[`, "prob")

# 4) Base con numero_de_cliente + promedio simple
ensamble <- pred_list[[1]][, .(numero_de_cliente)]
ensamble[, prob_ensamble := rowMeans(probs_mat)]

# 5) Nombres de salida con PARAM$experimento
archivo_ensamble      <- sprintf("ensamble_produccion_%s.csv", PARAM$experimento)
archivo_top11000      <- sprintf("ensamble_produccion_top11000_%s.csv", PARAM$experimento)

# 6) Guardar ensamble completo
fwrite(
  ensamble,
  file = archivo_ensamble
)

# 7) Ordenar de mayor a menor y quedarme con top 11000
top_n <- 11000L
top_ensamble <- ensamble[order(-prob_ensamble)][
  1:min(.N, top_n),
  .(numero_de_cliente)
]

# 8) Guardar solo lista de numero_de_cliente sin encabezado
fwrite(
  top_ensamble,
  file = archivo_top11000,
  col.names = FALSE
)
